# Make ASV taxa table (custom)

- From custom blast search xml file
- Analyze PR2 results, get top hits

### Import libraries and set directory location

In [1]:
import sys
import pandas as pd
import numpy as np
import glob
from Bio.Blast import NCBIXML   #to parse XML files

In [14]:
directory = '/Users/kpitz/github/GLOMICON/intercomparison/Merged_Datasets/'
prefix = 'GLOMICON'
marker = '18S'
blast_file = '/Users/kpitz/github/GLOMICON/intercomparison/Merged_Datasets/PR2/GLOMICON_PR2_version_5.1.0_blastn_041025.xml'
print(blast_file)
taxa_table = '/Users/kpitz/github/GLOMICON/intercomparison/Merged_Datasets/data/GLOMICON_taxa_blastnr.csv'
print(taxa_table)
seq_table = '/Users/kpitz/github/GLOMICON/intercomparison/Merged_Datasets/data/GLOMICON_seq_merged.csv'
print(seq_table)

/Users/kpitz/github/GLOMICON/intercomparison/Merged_Datasets/PR2/GLOMICON_PR2_version_5.1.0_blastn_041025.xml
/Users/kpitz/github/GLOMICON/intercomparison/Merged_Datasets/data/GLOMICON_taxa_blastnr.csv
/Users/kpitz/github/GLOMICON/intercomparison/Merged_Datasets/data/GLOMICON_seq_merged.csv


## Import Data

In [11]:
#sequence table
df = pd.read_csv(seq_table)
df = df.rename(columns={'Unnamed: 0':'ASV'})
df.set_index('ASV', inplace=True)
seq = df.copy()
seq.head()

# Dada2, MEGAN, Blastn results
df = pd.read_csv(taxa_table)
df = df.rename(columns={'Unnamed: 0':'ASV'})
df.set_index('ASV', inplace=True)
taxa = df.copy()
taxa.head()
df

,Domain,Kingdom,Phylum,Class,Order,Family,Genus,Species
ASV,,,,,,,,
ASV_1950,Archaea,NaN,unknown,unknown,unknown,unknown,unknown,archaeon
ASV_9581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ASV_63,Archaea,NaN,unknown,unknown,unknown,unknown,unknown,archaeon
ASV_3182,Archaea,NaN,unknown,unknown,unknown,unknown,unknown,archaeon
ASV_603,Archaea,NaN,unknown,unknown,unknown,unknown,unknown,archaeon
...,...,...,...,...,...,...,...,...
ASV_12927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ASV_12880,Eukaryota,NaN,Bacillariophyta,Bacillariophyceae,Mastogloiales,Achnanthaceae,Achnanthes,NaN
ASV_14103,Eukaryota,NaN,unknown,unknown,unknown,unknown,unknown,uncultured marine eukaryote


## Parse XML file to get top hits per ASV and percent ID and bitscore values

In [12]:
# Only save top 2% of bitscore hits to query
print('Blast XML file: ', blast_file)
hit_dict ={}  #Dictionary to store hits
result_handle =open(blast_file)
#parse blast records
blast_records= NCBIXML.parse(result_handle)
for blast_record in blast_records:
    query = blast_record.query
    hit_ID=0 #hit counter
    for alignment in blast_record.alignments:
        hit_ID+=1
        key = (query, hit_ID)
        hsp = alignment.hsps[0] #only look at top hsp per alignment to genbank sequence
        #set limits on evalue and bitscore and % Identity
        per_iden = hsp.identities/float(hsp.align_length)
        per_iden = (per_iden *100)
        per_iden = round(per_iden, 2)
        evalue = hsp.expect
        bitscore = int(hsp.score)
        # set bitscore limit for hits, within top 2%
        if hit_ID ==1:
            top_bitscore = bitscore
            min_bitscore = top_bitscore - (top_bitscore*.02)
        if bitscore < min_bitscore:
            #continue to next alignment if bitscore too low
            continue
        #alignment length
        align_len = hsp.align_length
        # save results to dictionary
        value = (evalue, bitscore, per_iden, alignment.hit_def, align_len)
        hit_dict[key]=value
print ('Done parsing ', blast_file)

Blast XML file:  /Users/kpitz/github/GLOMICON/intercomparison/Merged_Datasets/PR2/GLOMICON_PR2_version_5.1.0_blastn_041025.xml
Done parsing  /Users/kpitz/github/GLOMICON/intercomparison/Merged_Datasets/PR2/GLOMICON_PR2_version_5.1.0_blastn_041025.xml


In [13]:
#Save Blast Results to file
df = pd.DataFrame(hit_dict)
df=df.T
df.columns = ['eval','bitscore', '%ID', 'hit_def', 'align_len']
df.reset_index(inplace=True)
df = df.rename(columns={'level_0': 'ASV', 'level_1': 'Hit_number'})
# parse out PR2 information from 'hit_def'
print(df['hit_def'].iloc[0])
# domain, supergroup, division, subdivision, class, order, family, genus, species
col1 = ['PR2_ID', 'PR2_marker', 'location','nuc',
        'Domain','Supergroup','Division','Subdivision','Class', 'Order',
        'Family','Genus', 'Species']
#col2 = ['Kingdom', 'Phylum', 'Class', 'Order', 'Family','subfamily','Genus', 'Species']
# split PR2 columns
for i in range(len(col1)):
    df[col1[i]] = df['hit_def'].str.split('|').str[i]
# split taxonomy levels
# for i in range(len(col2)):
#     df[col2[i]] = df['PR2_taxonomy'].str.split(',').str[i]
df.set_index('ASV', inplace=True)
blast_results = df.copy()
blast_results.head()

KC488454.1.1667_U|18S_rRNA|nucleus|clone_HL5aSF04.03|Eukaryota|Haptista|Haptophyta|Haptophyta_X|Prymnesiophyceae|Phaeocystales|Phaeocystaceae|Phaeocystis|Phaeocystis_pouchetii


,Hit_number,eval,bitscore,%ID,hit_def,align_len,PR2_ID,PR2_marker,location,nuc,Domain,Supergroup,Division,Subdivision,Class,Order,Family,Genus,Species
ASV,,,,,,,,,,,,,,,,,,,
ASV_1,1,0.0,350,100.0,KC488454.1.1667_U|18S_rRNA|nucleus|clone_HL5aS...,350,KC488454.1.1667_U,18S_rRNA,nucleus,clone_HL5aSF04.03,Eukaryota,Haptista,Haptophyta,Haptophyta_X,Prymnesiophyceae,Phaeocystales,Phaeocystaceae,Phaeocystis,Phaeocystis_pouchetii
ASV_1,2,0.0,350,100.0,HM561166.1.964_U|18S_rRNA|nucleus|clone_CFL133...,350,HM561166.1.964_U,18S_rRNA,nucleus,clone_CFL133R09,Eukaryota,Haptista,Haptophyta,Haptophyta_X,Prymnesiophyceae,Phaeocystales,Phaeocystaceae,Phaeocystis,Phaeocystis_pouchetii
ASV_1,3,0.0,350,100.0,HM561167.1.839_U|18S_rRNA|nucleus|clone_CFL133...,350,HM561167.1.839_U,18S_rRNA,nucleus,clone_CFL133R10,Eukaryota,Haptista,Haptophyta,Haptophyta_X,Prymnesiophyceae,Phaeocystales,Phaeocystaceae,Phaeocystis,Phaeocystis_pouchetii
ASV_1,4,0.0,350,100.0,HM561164.1.955_U|18S_rRNA|nucleus|clone_CFL133...,350,HM561164.1.955_U,18S_rRNA,nucleus,clone_CFL133DB11,Eukaryota,Haptista,Haptophyta,Haptophyta_X,Prymnesiophyceae,Phaeocystales,Phaeocystaceae,Phaeocystis,Phaeocystis_pouchetii
ASV_1,5,0.0,350,100.0,HM561163.1.945_U|18S_rRNA|nucleus|clone_CFL133...,350,HM561163.1.945_U,18S_rRNA,nucleus,clone_CFL133DB10,Eukaryota,Haptista,Haptophyta,Haptophyta_X,Prymnesiophyceae,Phaeocystales,Phaeocystaceae,Phaeocystis,Phaeocystis_pouchetii


## Export blast results

In [19]:
# just top hits
df = blast_results.copy()
df = df.reset_index()
df = df.drop_duplicates('ASV')
df = df.set_index('ASV')
df.to_csv(directory+'PR2/PR2_blast_results_tophits.csv')
df

,Hit_number,eval,bitscore,%ID,hit_def,align_len,PR2_ID,PR2_marker,location,nuc,Domain,Supergroup,Division,Subdivision,Class,Order,Family,Genus,Species
ASV,,,,,,,,,,,,,,,,,,,
ASV_1,1,0.0,350,100.0,KC488454.1.1667_U|18S_rRNA|nucleus|clone_HL5aS...,350,KC488454.1.1667_U,18S_rRNA,nucleus,clone_HL5aSF04.03,Eukaryota,Haptista,Haptophyta,Haptophyta_X,Prymnesiophyceae,Phaeocystales,Phaeocystaceae,Phaeocystis,Phaeocystis_pouchetii
ASV_2,1,0.0,127,100.0,AJ535170.1.1763_U|18S_rRNA|nucleus|clone_p928|...,127,AJ535170.1.1763_U,18S_rRNA,nucleus,clone_p928,Eukaryota,TSAR,Stramenopiles,Gyrista,Mediophyceae,Thalassiosirales,Thalassiosiraceae,Conticribra,Conticribra_weissflogii
ASV_3,1,0.0,118,97.64,KJ961704.1.1788_U|18S_rRNA|nucleus|strain_52|E...,127,KJ961704.1.1788_U,18S_rRNA,nucleus,strain_52,Eukaryota,TSAR,Stramenopiles,Gyrista,Coscinodiscophyceae,Melosirales,Melosiraceae,Melosira,Melosira_sp.
ASV_4,1,0.0,127,100.0,GU373969.1.1789_U|18S_rRNA|nucleus|strain_CCMP...,127,GU373969.1.1789_U,18S_rRNA,nucleus,strain_CCMP1440,Eukaryota,TSAR,Stramenopiles,Gyrista,Bacillariophyceae,Bacillariales,Bacillariaceae,Pseudo-nitzschia,Pseudo-nitzschia_seriata
ASV_5,1,0.0,126,100.0,MN965778.1.1765_U|18S_rRNA|nucleus||Eukaryota|...,126,MN965778.1.1765_U,18S_rRNA,nucleus,,Eukaryota,TSAR,Alveolata,Dinoflagellata,Dinophyceae,Gymnodiniales,Kareniaceae,Shimiella,Shimiella_gracilenta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ASV_14543,1,0.0,357,97.9,KY979976.1.1705_U|18S_rRNA|nucleus||Eukaryota|...,381,KY979976.1.1705_U,18S_rRNA,nucleus,,Eukaryota,TSAR,Stramenopiles,Gyrista,Pelagophyceae,Pelagomonadales,Pelagomonadales_clade_A,Pelagomonadales_clade_A_X,Pelagomonadales_clade_A_X_sp.
ASV_14544,1,0.0,361,99.73,KR064016.1.709_U|18S_rRNA|nucleus|clone_St4_DC...,364,KR064016.1.709_U,18S_rRNA,nucleus,clone_St4_DCM_31-71_502f,Eukaryota,TSAR,Alveolata,Ciliophora,Oligohymenophorea,OLIGO5,OLIGO5_X,OLIGO5_XX,OLIGO5_XX_sp.
ASV_14545,1,0.0,368,100.0,KJ762992.1.1771_U|18S_rRNA|nucleus|clone_SGUH1...,368,KJ762992.1.1771_U,18S_rRNA,nucleus,clone_SGUH1368,Eukaryota,TSAR,Alveolata,Ciliophora,Spirotrichea,Oligotrichida,Strombidiidae,Strombidiidae_X,Strombidiidae_X_sp.


## Compare to blastn MEGAN results (nt)

In [15]:
df = blast_results.copy()
df = df.reset_index()
df = df.drop_duplicates('ASV')
df = df.set_index('ASV')
df = pd.concat([df, taxa], axis=1, keys=['PR2','MEGAN'])

levels = ['Class', 'Order', 'Family','Genus', 'Species']
for tax in levels:
    df[tax+'_match'] = 0
    df.loc[df['PR2', tax]==df['MEGAN', tax], tax+'_match'] = 1
    # 26245

cols = ['Class_match', 'Order_match', 'Family_match','Genus_match', 'Species_match']
df = df.sort_values(cols)

'''#Genus
df['genus_match'] = 0
df.loc[df['PR2', 'Genus']==df['MEGAN', 'Genus'], 'genus_match'] = 1
# 26245
df = df.loc[df['genus_match']==1]
#3953 genus
df = df.sort_values('genus_match')'''

#print(directory+'test.csv')
df.to_csv(directory+'test.csv')
df = df.loc[df['MEGAN','Phylum']=='Bacillariophyta']
df = df.sort_values([('PR2','Class'), ('PR2','Order'), ('PR2','Family'),('PR2','Genus'), ('PR2','Species')])
#df = df.drop_duplicates([('PR2','Species'), ('MEGAN', 'Species')])
#df['count'] = 1
#df = df.groupby([('PR2','Species'), ('MEGAN', 'Species')]).sum()
#df = df[['count']]
df.to_csv(directory+'PR2/test_diatom.csv')
print(directory+'PR2/test_diatom.csv')
df

/Users/kpitz/github/GLOMICON/intercomparison/Merged_Datasets/PR2/test_diatom.csv


PR2                       \
          Hit_number eval bitscore    %ID   
ASV                                         
ASV_4114         1.0  0.0       97  92.19   
ASV_12880        1.0  0.0      319   95.2   
ASV_4469         1.0  0.0       94  92.56   
ASV_972          1.0  0.0      127  100.0   
ASV_10097        1.0  0.0      124  99.21   
...              ...  ...      ...    ...   
ASV_431          NaN  NaN      NaN    NaN   
ASV_1577         NaN  NaN      NaN    NaN   
ASV_3465         NaN  NaN      NaN    NaN   
ASV_9410         NaN  NaN      NaN    NaN   
ASV_2514         NaN  NaN      NaN    NaN   

                                                                        \
                                                     hit_def align_len   
ASV                                                                      
ASV_4114   AJ535150.1.1787_U|18S_rRNA|nucleus|clone_p387|...       128   
ASV_12880  AJ535150.1.1787_U|18S_rRNA|nucleus|clone_p387|...       375   
ASV_4469   HQ655888.1.1730_U|18S_rRNA|nucleus||Eukaryota|...       121   
ASV_972    OR286793.1.1783_U|18S_rRNA|nucleus||Eukaryota|...       127   
ASV_10097  OR286793.1.1783_U|18S_rRNA|nucleus||Eukaryota|...       127   
...                                                      ...       ...   
ASV_431                                                  NaN       NaN   
ASV_1577                                                 NaN       NaN   
ASV_3465                                                 NaN       NaN   
ASV_9410                                                 NaN       NaN   
ASV_2514                                                 NaN       NaN   

                                                              ...  \
                      PR2_ID PR2_marker location         nuc  ...   
ASV                                                           ...   
ASV_4114   AJ535150.1.1787_U   18S_rRNA  nucleus  clone_p387  ...   
ASV_12880  AJ535150.1.1787_U   18S_rRNA  nucleus  clone_p387  ...   
ASV_4469   HQ655888.1.1730_U   18S_rRNA  nucleus              ...   
ASV_972    OR286793.1.1783_U   18S_rRNA  nucleus              ...   
ASV_10097  OR286793.1.1783_U   18S_rRNA  nucleus              ...   
...                      ...        ...      ...         ...  ...   
ASV_431                  NaN        NaN      NaN         NaN  ...   
ASV_1577                 NaN        NaN      NaN         NaN  ...   
ASV_3465                 NaN        NaN      NaN         NaN  ...   
ASV_9410                 NaN        NaN      NaN         NaN  ...   
ASV_2514                 NaN        NaN      NaN         NaN  ...   

                         MEGAN                                       \
                         Class             Order             Family   
ASV                                                                   
ASV_4114                   NaN               NaN                NaN   
ASV_12880    Bacillariophyceae     Mastogloiales      Achnanthaceae   
ASV_4469     Bacillariophyceae               NaN                NaN   
ASV_972      Bacillariophyceae     Bacillariales     Bacillariaceae   
ASV_10097    Bacillariophyceae     Bacillariales     Bacillariaceae   
...                        ...               ...                ...   
ASV_431           Mediophyceae     Cymatosirales     Cymatosiraceae   
ASV_1577   Coscinodiscophyceae  Stephanodiscales  Stephanodiscaceae   
ASV_3465   Coscinodiscophyceae        Paraliales        Paraliaceae   
ASV_9410   Coscinodiscophyceae   Chaetocerotales   Chaetocerotaceae   
ASV_2514   Coscinodiscophyceae       Melosirales       Melosiraceae   

                                                        Class_match  \
                    Genus                       Species               
ASV                                                                   
ASV_4114              NaN                           NaN           0   
ASV_12880      Achnanthes                           NaN           1   
ASV_4469              NaN       

In [16]:
df = blast_results.copy()
df = df.reset_index()
df = df.drop_duplicates('ASV')
df = df.set_index('ASV')
df = pd.concat([df, taxa], axis=1, keys=['PR2','MEGAN'])

levels = ['Class', 'Order', 'Family','Genus', 'Species']
for tax in levels:
    df[tax+'_match'] = 0
    df.loc[df['PR2', tax]==df['MEGAN', tax], tax+'_match'] = 1
    # 26245

cols = ['Class_match', 'Order_match', 'Family_match','Genus_match', 'Species_match']
df = df.sort_values(cols)

'''#Genus
df['genus_match'] = 0
df.loc[df['PR2', 'Genus']==df['MEGAN', 'Genus'], 'genus_match'] = 1
# 26245
df = df.loc[df['genus_match']==1]
#3953 genus
df = df.sort_values('genus_match')'''

print(directory+'PR2/test.csv')
df.to_csv(directory+'PR2/test.csv')
#df = df.loc[df['PR2','Class']=='Dinophyceae']
df = df.loc[df['MEGAN','Phylum']=='Bacillariophyta']
df = df.sort_values([('PR2','Class'), ('PR2','Order'), ('PR2','Family'),('PR2','Genus'), ('PR2','Species')])
#df = df.drop_duplicates([('PR2','Species'), ('MEGAN', 'Species')])
df['count'] = 1
df = df.groupby([('PR2','Family'), ('MEGAN', 'Family')]).sum()
df = df[['count']]
df.to_csv(directory+'PR2/test_Class.csv')
df

/Users/kpitz/github/GLOMICON/intercomparison/Merged_Datasets/test.csv


count
                                             
(PR2, Family)         (MEGAN, Family)        
Achnanthaceae         Achnanthaceae         1
Anaulaceae            Anaulaceae            2
Asterionellopsidaceae Fragilariaceae        2
Asterolampraceae      Asterolampraceae      4
Attheyaceae           Attheyaceae           4
...                                       ...
Thalassiosiraceae     Cymatosiraceae        3
                      Pleurosigmataceae     1
                      Stephanodiscaceae     1
                      Thalassiosiraceae   285
Triceratiaceae        Triceratiaceae        3

[93 rows x 1 columns]

In [17]:
df = blast_results.copy()
df = df.reset_index()
df = df.drop_duplicates('ASV')
df = df.set_index('ASV')
df = pd.concat([df, taxa], axis=1, keys=['PR2','MEGAN'])
df = df.loc[df['PR2','Genus']=='Pseudocalanus']

levels = ['Kingdom', 'Phylum', 'Class', 'Order', 'Family','Genus', 'Species']
for tax in levels:
    df[tax+'_match'] = 0
    df.loc[df['PR2', tax]==df['MEGAN', tax], tax+'_match'] = 1
    # 26245

cols = ['Kingdom_match', 'Phylum_match', 'Class_match', 'Order_match', 'Family_match','Genus_match', 'Species_match']
df = df.sort_values(cols)

'''#Genus
df['genus_match'] = 0
df.loc[df['PR2', 'Genus']==df['MEGAN', 'Genus'], 'genus_match'] = 1
# 26245
df = df.loc[df['genus_match']==1]
#3953 genus
df = df.sort_values('genus_match')'''

'''print(directory+'test.csv')
df.to_csv(directory+'test.csv')
df = df.loc[df['PR2','Class']=='Copepoda']
df = df.sort_values([('PR2','Class'), ('PR2','Order'), ('PR2','Family'),('PR2','Genus'), ('PR2','Species')])
#df = df.drop_duplicates([('PR2','Species'), ('MEGAN', 'Species')])
df['count'] = 1
df = df.groupby([('PR2','Species'), ('MEGAN', 'Species')]).sum()
df = df[['count']]
df.to_csv(directory+'test_copepoda.csv')'''
df = df.sort_values([('PR2','Class'), ('PR2','Order'), ('PR2','Family'),('PR2','Genus'), ('PR2','Species')])
df = df.loc[(df[('PR2', 'Species')]=='Pseudocalanus mimus')&(df[('MEGAN', 'Species')]!='Pseudocalanus mimus')]
df.to_csv(directory+'PR2/test_copepoda.csv')
df

KeyError: ('PR2', 'Kingdom')

In [33]:
df = blast_results.copy()
#df = df.loc[df['ASV']=='ASV_5']
df = df.drop_duplicates('ASV')
df = df.loc[df['%ID']<80]
#df= df.loc[df['bitscore']>(308 - (308*.02))]
df

,ASV,Hit_number,eval,bitscore,%ID,hit_def,align_len
38,ASV_3,1,0.0,110,79.73,"BHNHM212-24|COI-5P|Afghanistan|Animalia,Arthro...",291
49,ASV_5,1,0.0,113,79.52,"GMQQT008-18|COI-5P|Australia|Animalia,Arthropo...",293
161,ASV_20,1,0.0,113,79.52,"GMQQT008-18|COI-5P|Australia|Animalia,Arthropo...",293
163,ASV_23,1,0.0,105,78.69,"CRBOF22983-24|COI-5P|Costa Rica|Animalia,Arthr...",291
367,ASV_27,1,0.0,122,79.81,"GBAAY62677-24|COI-5P|Unrecoverable|Mixture,Uns...",312
...,...,...,...,...,...,...,...
272978,ASV_269846,1,0.0,101,79.21,"NBINS004-14|COI-5P|United States|Animalia,Arth...",279
274072,ASV_270224,1,0.0,97,77.59,"POMS7650-24|COI-5P|United Kingdom|Animalia,Art...",299
277208,ASV_272913,1,0.0,102,78.35,"CRALC29584-21|COI-5P|Costa Rica|Animalia,Arthr...",291
277375,ASV_273151,1,0.0,98,78.16,"GBAAY56519-24|COI-5P|Unrecoverable|Protista,Ha...",293


In [42]:
df = blast_results.copy()
#df = df.loc[df['ASV']=='ASV_5']
df = df.drop_duplicates('ASV')
df = df.loc[df['%ID']<80]
print(df['hit_def'].iloc[0])
col1 = ['PR2_ID', 'PR2_marker', 'PR2_location', 'PR2_taxonomy']
col2 = ['Kingdom', 'Phylum', 'Class', 'Order', 'Family','subfamily','Genus', 'Species']
for i in range(len(col1)):
    df[col1[i]] = df['hit_def'].str.split('|').str[i]

for i in range(len(col2)):
    df[col2[i]] = df['PR2_taxonomy'].str.split(',').str[i]
df

BHNHM212-24|COI-5P|Afghanistan|Animalia,Arthropoda,Insecta,Lepidoptera,Noctuidae,Noctuinae,Caradrina,Caradrina fergana,None


,ASV,Hit_number,eval,bitscore,%ID,hit_def,align_len,BOLD_ID,BOLD_marker,BOLD_location,BOLD_taxonomy,Kingdom,Phylum,Class,Order,Family,subfamily,Genus,Species
38,ASV_3,1,0.0,110,79.73,"BHNHM212-24|COI-5P|Afghanistan|Animalia,Arthro...",291,BHNHM212-24,COI-5P,Afghanistan,"Animalia,Arthropoda,Insecta,Lepidoptera,Noctui...",Animalia,Arthropoda,Insecta,Lepidoptera,Noctuidae,Noctuinae,Caradrina,Caradrina fergana
49,ASV_5,1,0.0,113,79.52,"GMQQT008-18|COI-5P|Australia|Animalia,Arthropo...",293,GMQQT008-18,COI-5P,Australia,"Animalia,Arthropoda,Insecta,Diptera,Rhagionida...",Animalia,Arthropoda,Insecta,Diptera,Rhagionidae,None,None,None
161,ASV_20,1,0.0,113,79.52,"GMQQT008-18|COI-5P|Australia|Animalia,Arthropo...",293,GMQQT008-18,COI-5P,Australia,"Animalia,Arthropoda,Insecta,Diptera,Rhagionida...",Animalia,Arthropoda,Insecta,Diptera,Rhagionidae,None,None,None
163,ASV_23,1,0.0,105,78.69,"CRBOF22983-24|COI-5P|Costa Rica|Animalia,Arthr...",291,CRBOF22983-24,COI-5P,Costa Rica,"Animalia,Arthropoda,Insecta,Diptera,Ceratopogo...",Animalia,Arthropoda,Insecta,Diptera,Ceratopogonidae,None,None,None
367,ASV_27,1,0.0,122,79.81,"GBAAY62677-24|COI-5P|Unrecoverable|Mixture,Uns...",312,GBAAY62677-24,COI-5P,Unrecoverable,"Mixture,Unspecified,None,None,None,None,None,N...",Mixture,Unspecified,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272978,ASV_269846,1,0.0,101,79.21,"NBINS004-14|COI-5P|United States|Animalia,Arth...",279,NBINS004-14,COI-5P,United States,"Animalia,Arthropoda,Insecta,Coleoptera,Tenebri...",Animalia,Arthropoda,Insecta,Coleoptera,Tenebrionidae,Tenebrioninae,Cerenopus,Cerenopus concolor
274072,ASV_270224,1,0.0,97,77.59,"POMS7650-24|COI-5P|United Kingdom|Animalia,Art...",299,POMS7650-24,COI-5P,United Kingdom,"Animalia,Arthropoda,Insecta,Diptera,Tachinidae...",Animalia,Arthropoda,Insecta,Diptera,Tachinidae,Dexiinae,Eriothrix,Eriothrix rufomaculata
277208,ASV_272913,1,0.0,102,78.35,"CRALC29584-21|COI-5P|Costa Rica|Animalia,Arthr...",291,CRALC29584-21,COI-5P,Costa Rica,"Animalia,Arthropoda,Insecta,Diptera,Ceratopogo...",Animalia,Arthropoda,Insecta,Diptera,Ceratopogonidae,None,None,None
277375,ASV_273151,1,0.0,98,78.16,"GBAAY56519-24|COI-5P|Unrecoverable|Protista,Ha...",293,GBAAY56519-24,COI-5P,Unrecoverable,"Protista,Haptophyta,Prymnesiophyceae,Isochrysi...",Protista,Haptophyta,Prymnesiophyceae,Isochrysidales,None,None,None,None


In [ ]:
#Save Blast Results to file
df = pd.DataFrame(hit_dict)
df=df.T
df.columns = ['eval','bitscore', '%ID', 'hit_def', 'align_len']
df.reset_index(inplace=True)
df = df.rename(columns={'level_0': 'ASV', 'level_1': 'Hit_number'})


#Keep the best hit of each category of hit
df = df.drop_duplicates(subset=['genus', 'species', 'OTU'], keep='first')
#Sort by OTU number
df['OTU_Number']= df['OTU'].str.split('_').str[1]
df['OTU_Number'] = df['OTU_Number'].astype(int)
df.sort_values(['OTU_Number','Hit_number'], inplace=True)
df.drop('OTU_Number', axis=1, inplace=True)
df.set_index(['OTU', 'Hit_number'], inplace = True)
outfile = filename.replace('.xml','_TopBLASThits.csv')
print(outfile)
df.to_csv(outfile)
xml_dfs.append(df)

In [ ]:



#Save Blast Results to file
df = pd.DataFrame(hit_dict)
df=df.T
df.columns = ['eval','bitscore', '%ID', 'genus', 'species', 'hit_def', 'align_len']
df.reset_index(inplace=True)
df = df.rename(columns={'level_0': 'OTU', 'level_1': 'Hit_number'})
#Keep the best hit of each category of hit
df = df.drop_duplicates(subset=['genus', 'species', 'OTU'], keep='first')
#Sort by OTU number
df['OTU_Number']= df['OTU'].str.split('_').str[1]
df['OTU_Number'] = df['OTU_Number'].astype(int)
df.sort_values(['OTU_Number','Hit_number'], inplace=True)
df.drop('OTU_Number', axis=1, inplace=True)
df.set_index(['OTU', 'Hit_number'], inplace = True)
outfile = filename.replace('.xml','_TopBLASThits.csv')
print(outfile)
df.to_csv(outfile)
xml_dfs.append(df)










    #parse filename and directory location
    print('Blast XML file: ', filename)
    #directory = filename.split('/')[:-2]
    #string_s= '/'
    #directory=string_s.join(directory)
    #directory = directory +'/'
    #print('directory', directory)
    genus=[]
    species=[]
    query=[]
    hit_dict ={}  #Dictionary to store hits
    result_handle =open(filename)
    #Phix names to filter out - added for diversity in sequencing process
    taxon = ['PhiX', 'phiX']
    #parse blast records
    blast_records= NCBIXML.parse(result_handle)
    #Locations to store PhiX hits
    taxon_A =[]
    taxon_B =[]
    query_A =[]
    query_B =[]
    for blast_record in blast_records:
        query = blast_record.query
        genus = str(taxa_dict[query][-2]) #look up genus assignment
        species = str(taxa_dict[query][-1]) #look up species assignment
        hit_ID=0 #hit counter
        for alignment in blast_record.alignments:
            hit_ID+=1
            key = (query, hit_ID)
            hsp = alignment.hsps[0] #only look at top hsp per alignment to genbank sequence
            #set limits on evalue and bitscore and % Identity
            per_iden = hsp.identities/float(hsp.align_length)
            per_iden = (per_iden *100)
            per_iden = round(per_iden, 2)
            evalue = hsp.expect
            bitscore = int(hsp.score)
            #alignment length
            align_len = hsp.align_length
            #Check if MEGAN assigned ASV at species level;look for species in hit ID if yes
            if species in ['s_', 'no_hit', 'unassigned']:
                species_h = 'species_unassigned'
            else:
                if species in alignment.hit_def:
                    species_h = species
                else:
                    species_h ='Not_Found'
            #Check if MEGAN assigned ASV at genus level;look for species in hit ID if yes
            if genus in ['g_', 'no_hit', 'unassigned']:
                genus_h = 'genus_unassigned'
            else:
                if genus in alignment.hit_def:
                    genus_h = genus
                else:
                    genus_h ='Not_Found'
            value = (evalue, bitscore, per_iden, genus_h, species_h, alignment.hit_def, align_len)
            hit_dict[key]=value
            genus_h='' #reset variables
            species_h=''#reset variables
            #Look for PhiX hits
            if taxon[0] in alignment.title:
                #print (alignment.title)
                taxon_A.append(blast_record.query)
                query_A.append(alignment.title)
            if taxon[1] in alignment.title:
                #print (alignment.title)
                taxon_B.append(blast_record.query)
                query_B.append(alignment.title)
        #break
    print ('Done parsing ', filename)

    #Save Blast Results to file
    df = pd.DataFrame(hit_dict)
    df=df.T
    df.columns = ['eval','bitscore', '%ID', 'genus', 'species', 'hit_def', 'align_len']
    df.reset_index(inplace=True)
    df = df.rename(columns={'level_0': 'OTU', 'level_1': 'Hit_number'})
    #Keep the best hit of each category of hit
    df = df.drop_duplicates(subset=['genus', 'species', 'OTU'], keep='first')
    #Sort by OTU number
    df['OTU_Number']= df['OTU'].str.split('_').str[1]
    df['OTU_Number'] = df['OTU_Number'].astype(int)
    df.sort_values(['OTU_Number','Hit_number'], inplace=True)
    df.drop('OTU_Number', axis=1, inplace=True)
    df.set_index(['OTU', 'Hit_number'], inplace = True)
    outfile = filename.replace('.xml','_TopBLASThits.csv')
    print(outfile)
    df.to_csv(outfile)
    xml_dfs.append(df)
    
    

In [81]:
# Parse genbank xml files (from mbon-master workers, 6 total)

'''# COI limits:
#Species limits
#bitscore_sp = int(sys.argv[1])
#per_ID_sp = int(sys.argv[2])
bitscore_sp=400
per_ID_sp=97
print('Species bitscore limit: ', bitscore_sp, ', Percent Identity Limit: ', per_ID_sp)
#Genus limits
#bitscore_gn = int(sys.argv[3])
#per_ID_gn = int(sys.argv[4])
bitscore_gn=350
per_ID_gn=95
print('Genus bitscore limit: ', bitscore_gn, ', Percent Identity Limit: ', per_ID_gn)'''

#Taxa Table
taxa_tab = pd.concat([megan, seq], axis=1)
taxa_tab = taxa_tab[levels]

#Create dictionary of ASV ID to taxonomy
taxa_dict = taxa_tab.T.to_dict('list')


#print(directory)
file_loc = '/Users/kpitz/Projects/custom_blast/Blast_VM_output/*xml'

files = glob.glob(file_loc)
# Parse through XML files
xml_dfs = []  #will store results from each xml file
for filename in files:
    #parse filename and directory location
    print('Blast XML file: ', filename)
    #directory = filename.split('/')[:-2]
    #string_s= '/'
    #directory=string_s.join(directory)
    #directory = directory +'/'
    #print('directory', directory)
    genus=[]
    species=[]
    query=[]
    hit_dict ={}  #Dictionary to store hits
    result_handle =open(filename)
    #Phix names to filter out - added for diversity in sequencing process
    taxon = ['PhiX', 'phiX']
    #parse blast records
    blast_records= NCBIXML.parse(result_handle)
    #Locations to store PhiX hits
    taxon_A =[]
    taxon_B =[]
    query_A =[]
    query_B =[]
    for blast_record in blast_records:
        query = blast_record.query
        genus = str(taxa_dict[query][-2]) #look up genus assignment
        species = str(taxa_dict[query][-1]) #look up species assignment
        hit_ID=0 #hit counter
        for alignment in blast_record.alignments:
            hit_ID+=1
            key = (query, hit_ID)
            hsp = alignment.hsps[0] #only look at top hsp per alignment to genbank sequence
            #set limits on evalue and bitscore and % Identity
            per_iden = hsp.identities/float(hsp.align_length)
            per_iden = (per_iden *100)
            per_iden = round(per_iden, 2)
            evalue = hsp.expect
            bitscore = int(hsp.score)
            #alignment length
            align_len = hsp.align_length
            #Check if MEGAN assigned ASV at species level;look for species in hit ID if yes
            if species in ['s_', 'no_hit', 'unassigned']:
                species_h = 'species_unassigned'
            else:
                if species in alignment.hit_def:
                    species_h = species
                else:
                    species_h ='Not_Found'
            #Check if MEGAN assigned ASV at genus level;look for species in hit ID if yes
            if genus in ['g_', 'no_hit', 'unassigned']:
                genus_h = 'genus_unassigned'
            else:
                if genus in alignment.hit_def:
                    genus_h = genus
                else:
                    genus_h ='Not_Found'
            value = (evalue, bitscore, per_iden, genus_h, species_h, alignment.hit_def, align_len)
            hit_dict[key]=value
            genus_h='' #reset variables
            species_h=''#reset variables
            #Look for PhiX hits
            if taxon[0] in alignment.title:
                #print (alignment.title)
                taxon_A.append(blast_record.query)
                query_A.append(alignment.title)
            if taxon[1] in alignment.title:
                #print (alignment.title)
                taxon_B.append(blast_record.query)
                query_B.append(alignment.title)
        #break
    print ('Done parsing ', filename)

    #Save Blast Results to file
    df = pd.DataFrame(hit_dict)
    df=df.T
    df.columns = ['eval','bitscore', '%ID', 'genus', 'species', 'hit_def', 'align_len']
    df.reset_index(inplace=True)
    df = df.rename(columns={'level_0': 'OTU', 'level_1': 'Hit_number'})
    #Keep the best hit of each category of hit
    df = df.drop_duplicates(subset=['genus', 'species', 'OTU'], keep='first')
    #Sort by OTU number
    df['OTU_Number']= df['OTU'].str.split('_').str[1]
    df['OTU_Number'] = df['OTU_Number'].astype(int)
    df.sort_values(['OTU_Number','Hit_number'], inplace=True)
    df.drop('OTU_Number', axis=1, inplace=True)
    df.set_index(['OTU', 'Hit_number'], inplace = True)
    outfile = filename.replace('.xml','_TopBLASThits.csv')
    print(outfile)
    df.to_csv(outfile)
    xml_dfs.append(df)
    
    

Species bitscore limit:  400 , Percent Identity Limit:  97
Genus bitscore limit:  350 , Percent Identity Limit:  95
Blast XML file:  /Users/kpitz/Projects/custom_blast/Blast_VM_output/mbon-worker3__.xml
Done parsing  /Users/kpitz/Projects/custom_blast/Blast_VM_output/mbon-worker3__.xml
/Users/kpitz/Projects/custom_blast/Blast_VM_output/mbon-worker3___TopBLASThits.csv
Blast XML file:  /Users/kpitz/Projects/custom_blast/Blast_VM_output/mbon-worker1__.xml
Done parsing  /Users/kpitz/Projects/custom_blast/Blast_VM_output/mbon-worker1__.xml
/Users/kpitz/Projects/custom_blast/Blast_VM_output/mbon-worker1___TopBLASThits.csv
Blast XML file:  /Users/kpitz/Projects/custom_blast/Blast_VM_output/mbon-worker5__.xml
Done parsing  /Users/kpitz/Projects/custom_blast/Blast_VM_output/mbon-worker5__.xml
/Users/kpitz/Projects/custom_blast/Blast_VM_output/mbon-worker5___TopBLASThits.csv
Blast XML file:  /Users/kpitz/Projects/custom_blast/Blast_VM_output/mbon-worker2__.xml
Done parsing  /Users/kpitz/Projects

## Get df of sequence data

In [48]:
input_file = directory + 'Flyer2018_COI_seq_merged.fasta'
# fasta file to dataframe:

#From fasta file create pandas df of ASV and sequence
def from_fasta_to_df(file):
    #print(file)
    with open(file) as f:
        Ids=[]
        seqs =[]
        for strline in f:
            if strline[0]=='>':
                Ids.append(strline[1:].strip())
            else:
                seqs.append(strline.strip())
    print('Number of Ids:',len(Ids))
    print('Number of Seqs:',len(seqs))
    seq_dict = dict(zip(Ids, seqs))
    #make pandas df
    df= pd.DataFrame.from_dict(seq_dict,orient='index', columns=['sequence'])
    return df

df = from_fasta_to_df(input_file)
seq = df.copy()
seq.head()

Number of Ids: 15713
Number of Seqs: 15713


,sequence
ASV_1,TCTAGCAGGGATTCAAGCTCATTCAGGAGGTTCTGTTGATTTAGCA...
ASV_2,TCTAGCAGGGATTCAAGCTCATTCAGGAGGTTCTGTTGATTTAGCA...
ASV_3,TTAAGAATAAATATCGCCCATTCAGGCCCATCTGTCGATTTTGCTA...
ASV_4,TCTAGCAGGGATTCAAGCTCATTCAGGAGGTTCTGTTGATTTAGCA...
ASV_5,TCTAGCAGGGATTCAAGCTCATTCAGGAGGTTCTGTTGATTTAGCA...


## Parse MEGAN results from genbank blastn

In [61]:
#K metazoa Kingdom
levels = ['Domain', 'Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']
df = pd.read_csv(taxa_table_file, sep="\t", header=None, names =['ASV', 'taxonomy'])

#print(df['taxonomy'][405])
# extract all levels present in the taxonomy:
df.set_index('ASV', inplace=True)
#df['test'] = df['taxonomy'].str.split(';')
#df['len'] = df['test'].str.len()
df['Domain'] = df['taxonomy'].str.extract(r'\[D\] ([^;]*);.*')
df['Kingdom'] = df['taxonomy'].str.extract(r'\[K\] ([^;]*);.*')
df['Phylum'] = df['taxonomy'].str.extract(r'\[P\] ([^;]*);.*')
df['Class'] = df['taxonomy'].str.extract(r'\[C\] ([^;]*);.*')
df['Order'] = df['taxonomy'].str.extract(r'\[O\] ([^;]*);.*')
df['Family'] = df['taxonomy'].str.extract(r'\[F\] ([^;]*);.*')
df['Genus'] = df['taxonomy'].str.extract(r'\[G\] ([^;]*);.*')
df['Species'] = df['taxonomy'].str.extract(r'\[S\] ([^;]*);.*')

df = df.sort_values(levels)

# MEGAN already exports 'unknown' term but is inconsistent for some terms, like 'Kingdom'
#df = df.fillna('not assigned')
# if no taxonomy:
for level in levels:
    df.loc[df['taxonomy'].isna(), level] = 'not assigned'

#df = df.dropna(how='all')
#df.to_csv('/Users/kpitz/Documents/test.csv')
megan = df.copy()
megan.head()

[D] Eukaryota; [K] Metazoa; [P] Arthropoda; [C] Insecta; [O] Diptera;


,taxonomy,Domain,Kingdom,Phylum,Class,Order,Family,Genus,Species
ASV,,,,,,,,,
ASV_20618,[D] Archaea; [P] Thaumarchaeota;,Archaea,NaN,Thaumarchaeota,NaN,NaN,NaN,NaN,NaN
ASV_73595,[D] Bacteria; [P] Bacteroidetes; [C] Flavobact...,Bacteria,NaN,Bacteroidetes,Flavobacteriia,Flavobacteriales,Flavobacteriaceae,Cellulophaga,NaN
ASV_2609,[D] Bacteria; [P] Bacteroidetes; [C] Flavobact...,Bacteria,NaN,Bacteroidetes,Flavobacteriia,Flavobacteriales,Flavobacteriaceae,Cellulophaga,NaN
ASV_10045,[D] Bacteria; [P] Bacteroidetes; [C] Flavobact...,Bacteria,NaN,Bacteroidetes,Flavobacteriia,Flavobacteriales,Flavobacteriaceae,Cellulophaga,NaN
ASV_21215,[D] Bacteria; [P] Bacteroidetes; [C] Flavobact...,Bacteria,NaN,Bacteroidetes,Flavobacteriia,Flavobacteriales,Flavobacteriaceae,Flavobacterium,Flavobacterium sp. F-29


## Parse genbank XML files to get percent ID and bitscore values

In [81]:
# Parse genbank xml files (from mbon-master workers, 6 total)

# COI limits:
#Species limits
#bitscore_sp = int(sys.argv[1])
#per_ID_sp = int(sys.argv[2])
bitscore_sp=400
per_ID_sp=97
print('Species bitscore limit: ', bitscore_sp, ', Percent Identity Limit: ', per_ID_sp)
#Genus limits
#bitscore_gn = int(sys.argv[3])
#per_ID_gn = int(sys.argv[4])
bitscore_gn=350
per_ID_gn=95
print('Genus bitscore limit: ', bitscore_gn, ', Percent Identity Limit: ', per_ID_gn)

#Taxa Table
taxa_tab = pd.concat([megan, seq], axis=1)
taxa_tab = taxa_tab[levels]

#Create dictionary of ASV ID to taxonomy
taxa_dict = taxa_tab.T.to_dict('list')


#print(directory)
file_loc = '/Users/kpitz/Projects/custom_blast/Blast_VM_output/*xml'

files = glob.glob(file_loc)
# Parse through XML files
xml_dfs = []  #will store results from each xml file
for filename in files:
    #parse filename and directory location
    print('Blast XML file: ', filename)
    #directory = filename.split('/')[:-2]
    #string_s= '/'
    #directory=string_s.join(directory)
    #directory = directory +'/'
    #print('directory', directory)
    genus=[]
    species=[]
    query=[]
    hit_dict ={}  #Dictionary to store hits
    result_handle =open(filename)
    #Phix names to filter out - added for diversity in sequencing process
    taxon = ['PhiX', 'phiX']
    #parse blast records
    blast_records= NCBIXML.parse(result_handle)
    #Locations to store PhiX hits
    taxon_A =[]
    taxon_B =[]
    query_A =[]
    query_B =[]
    for blast_record in blast_records:
        query = blast_record.query
        genus = str(taxa_dict[query][-2]) #look up genus assignment
        species = str(taxa_dict[query][-1]) #look up species assignment
        hit_ID=0 #hit counter
        for alignment in blast_record.alignments:
            hit_ID+=1
            key = (query, hit_ID)
            hsp = alignment.hsps[0] #only look at top hsp per alignment to genbank sequence
            #set limits on evalue and bitscore and % Identity
            per_iden = hsp.identities/float(hsp.align_length)
            per_iden = (per_iden *100)
            per_iden = round(per_iden, 2)
            evalue = hsp.expect
            bitscore = int(hsp.score)
            #alignment length
            align_len = hsp.align_length
            #Check if MEGAN assigned ASV at species level;look for species in hit ID if yes
            if species in ['s_', 'no_hit', 'unassigned']:
                species_h = 'species_unassigned'
            else:
                if species in alignment.hit_def:
                    species_h = species
                else:
                    species_h ='Not_Found'
            #Check if MEGAN assigned ASV at genus level;look for species in hit ID if yes
            if genus in ['g_', 'no_hit', 'unassigned']:
                genus_h = 'genus_unassigned'
            else:
                if genus in alignment.hit_def:
                    genus_h = genus
                else:
                    genus_h ='Not_Found'
            value = (evalue, bitscore, per_iden, genus_h, species_h, alignment.hit_def, align_len)
            hit_dict[key]=value
            genus_h='' #reset variables
            species_h=''#reset variables
            #Look for PhiX hits
            if taxon[0] in alignment.title:
                #print (alignment.title)
                taxon_A.append(blast_record.query)
                query_A.append(alignment.title)
            if taxon[1] in alignment.title:
                #print (alignment.title)
                taxon_B.append(blast_record.query)
                query_B.append(alignment.title)
        #break
    print ('Done parsing ', filename)

    #Save Blast Results to file
    df = pd.DataFrame(hit_dict)
    df=df.T
    df.columns = ['eval','bitscore', '%ID', 'genus', 'species', 'hit_def', 'align_len']
    df.reset_index(inplace=True)
    df = df.rename(columns={'level_0': 'OTU', 'level_1': 'Hit_number'})
    #Keep the best hit of each category of hit
    df = df.drop_duplicates(subset=['genus', 'species', 'OTU'], keep='first')
    #Sort by OTU number
    df['OTU_Number']= df['OTU'].str.split('_').str[1]
    df['OTU_Number'] = df['OTU_Number'].astype(int)
    df.sort_values(['OTU_Number','Hit_number'], inplace=True)
    df.drop('OTU_Number', axis=1, inplace=True)
    df.set_index(['OTU', 'Hit_number'], inplace = True)
    outfile = filename.replace('.xml','_TopBLASThits.csv')
    print(outfile)
    df.to_csv(outfile)
    xml_dfs.append(df)
    
    

Species bitscore limit:  400 , Percent Identity Limit:  97
Genus bitscore limit:  350 , Percent Identity Limit:  95
Blast XML file:  /Users/kpitz/Projects/custom_blast/Blast_VM_output/mbon-worker3__.xml
Done parsing  /Users/kpitz/Projects/custom_blast/Blast_VM_output/mbon-worker3__.xml
/Users/kpitz/Projects/custom_blast/Blast_VM_output/mbon-worker3___TopBLASThits.csv
Blast XML file:  /Users/kpitz/Projects/custom_blast/Blast_VM_output/mbon-worker1__.xml
Done parsing  /Users/kpitz/Projects/custom_blast/Blast_VM_output/mbon-worker1__.xml
/Users/kpitz/Projects/custom_blast/Blast_VM_output/mbon-worker1___TopBLASThits.csv
Blast XML file:  /Users/kpitz/Projects/custom_blast/Blast_VM_output/mbon-worker5__.xml
Done parsing  /Users/kpitz/Projects/custom_blast/Blast_VM_output/mbon-worker5__.xml
/Users/kpitz/Projects/custom_blast/Blast_VM_output/mbon-worker5___TopBLASThits.csv
Blast XML file:  /Users/kpitz/Projects/custom_blast/Blast_VM_output/mbon-worker2__.xml
Done parsing  /Users/kpitz/Projects

In [98]:
#Look at all XML Files together

df= pd.concat(xml_dfs, axis=0)

# Begin filtering Taxonomic Annotations by Species and Genus limits
df[['eval', 'bitscore', '%ID', 'align_len']] = df[['eval', 'bitscore', '%ID', 'align_len']].apply(pd.to_numeric)
#Get top hits for taxa annotation assigned by MEGAN
df = df.reset_index().set_index('OTU')
df = df.join(taxa_tab, how='left')   #Join blast results with taxa table; anything without a blast hit won't join
df.reset_index(inplace=True)
df=df.sort_values(['index', 'Hit_number'])
df.set_index(['index', 'Hit_number'], inplace=True)

#Need to limit by top hit that includes genus and species, or just genus if that's all that's available
#Or if neither of them are available, set g_ and s_
#need to sort df by these levels; higher number, less specific assignment
df['G_lev']= 1 # 1 = assigned at that level; start out with everything =1
df['S_lev']= 1 # 1 = assigned at that level; start out with everything =1
# Now find unassigned ASVs:
df.loc[df['genus'] == 'genus_unassigned' , 'G_lev'] = 3
df.loc[df['genus'] == 'Not_Found' , 'G_lev'] = 2
df.loc[df['species'] == 'species_unassigned' , 'S_lev'] = 3
df.loc[df['species'] == 'Not_Found' , 'S_lev'] = 2

df = df.sort_values(['S_lev', 'G_lev'])
#Now just want to take the top hit for each OTU.
df=df.reset_index()
df = df.drop_duplicates('index', keep='first')
df.drop(['G_lev', 'S_lev'], axis=1, inplace=True)

#check no duplicates, one entry per OTU
dups = df.reset_index().duplicated(subset=['index'])
if dups.unique() != [False]:
    print('Error!! Duplicate OTUs in Table. Something went wrong.')


#Limit by percent ID
df.loc[df['%ID'] < per_ID_gn , 'Genus'] = "g_"
df.loc[df['%ID'] < per_ID_sp , 'Species'] = "s_"
#Limit by bitscore
df.loc[df['bitscore'] < bitscore_gn , 'Genus'] = "g_"
df.loc[df['bitscore'] < bitscore_sp , 'Species'] = "s_"

#sort by ASV number and hit number
df.reset_index(inplace=True)
df['OTU_Number']= df['index'].str.split('_').str[1]
df['OTU_Number'] = df['OTU_Number'].astype(int)
df.sort_values(['OTU_Number','Hit_number'], inplace=True)
df.set_index(['index','Hit_number'], inplace=True)

#Create New Taxa Table from selection
taxa_lim = df[levels]  #limit columns to taxonomy
taxa_lim = taxa_lim.reset_index().drop('Hit_number', axis=1)
taxa_lim = taxa_lim.rename(columns = {'index':'ASV'})
taxa_lim.set_index('ASV', inplace=True)
taxa_lim

,Domain,Kingdom,Phylum,Class,Order,Family,Genus,Species
ASV,,,,,,,,
ASV_1,Eukaryota,NaN,Haptophyta,unknown,Isochrysidales,Noelaerhabdaceae,NaN,NaN
ASV_2,Eukaryota,NaN,Haptophyta,unknown,Isochrysidales,Noelaerhabdaceae,NaN,NaN
ASV_3,Eukaryota,Metazoa,Arthropoda,NaN,NaN,NaN,g_,s_
ASV_4,Eukaryota,NaN,Haptophyta,unknown,Isochrysidales,Noelaerhabdaceae,NaN,NaN
ASV_5,Eukaryota,NaN,Haptophyta,unknown,Isochrysidales,Noelaerhabdaceae,NaN,NaN
...,...,...,...,...,...,...,...,...
ASV_138295,Eukaryota,NaN,unknown,Phaeophyceae,Ectocarpales,Chordariaceae,g_,s_
ASV_138351,Eukaryota,NaN,Oomycota,unknown,Saprolegniales,Saprolegniaceae,g_,s_
ASV_138768,Eukaryota,NaN,NaN,NaN,NaN,NaN,g_,s_


In [99]:
directory = '/Users/kpitz/Projects/custom_blast/'
outname = directory + 'Filtered_ASV_taxa_table.csv'
print(outname)
taxa_lim.to_csv(outname)

#Get average %ID of genus and species corrected ASVs:
print('XXXX  Check Stats of hits that have been filtered: XXXX')
df_g = df.loc[df['Genus']=='g_']
print('Mean %ID of ASVs with g_:',df_g['%ID'].mean())
print('Mean bitscore of ASVs with g_:',df_g['bitscore'].mean())
df_s = df.loc[df['Species']=='s_']
df_s = df_s.loc[df_s['Genus']!='g_']
print('Mean %ID of ASVs with s_:',df_s['%ID'].mean())
print('Mean bitscore of ASVs with s_:',df_s['bitscore'].mean())

df.drop('OTU_Number', axis=1, inplace=True)
df.drop('level_0', axis=1, inplace=True)
outname = directory + 'ASV_tophit_Ftaxonomy.csv'
print(outname)
df.to_csv(outname)
tophit_Ftaxonomy = df.copy()
df

/Users/kpitz/Projects/custom_blast/Filtered_ASV_taxa_table.csv
XXXX  Check Stats of hits that have been filtered: XXXX
Mean %ID of ASVs with g_: 84.05301163050773
Mean bitscore of ASVs with g_: 350.2860657571013
Mean %ID of ASVs with s_: 96.00779761904762
Mean bitscore of ASVs with s_: 538.202380952381
/Users/kpitz/Projects/custom_blast/ASV_tophit_Ftaxonomy.csv


,,eval,bitscore,%ID,genus,species,hit_def,align_len,Domain,Kingdom,Phylum,Class,Order,Family,Genus,Species
index,Hit_number,,,,,,,,,,,,,,,
ASV_1,1,7.533310e-154,616,100.00,Not_Found,Not_Found,Gephyrocapsa muellerae strain RCC3370 mitochon...,308,Eukaryota,NaN,Haptophyta,unknown,Isochrysidales,Noelaerhabdaceae,NaN,NaN
ASV_2,1,7.533310e-154,616,100.00,Not_Found,Not_Found,Emiliania huxleyi strain RCC4002 mitochondrion...,308,Eukaryota,NaN,Haptophyta,unknown,Isochrysidales,Noelaerhabdaceae,NaN,NaN
ASV_3,1,2.999550e-70,309,80.13,Not_Found,Not_Found,Oncaea mediterranea voucher P0134 cytochrome o...,307,Eukaryota,Metazoa,Arthropoda,NaN,NaN,NaN,g_,s_
ASV_4,1,7.533310e-154,616,100.00,Not_Found,Not_Found,Gephyrocapsa parvula strain RCC4034 mitochondr...,308,Eukaryota,NaN,Haptophyta,unknown,Isochrysidales,Noelaerhabdaceae,NaN,NaN
ASV_5,1,3.203250e-152,611,99.68,Not_Found,Not_Found,Gephyrocapsa parvula strain RCC4034 mitochondr...,308,Eukaryota,NaN,Haptophyta,unknown,Isochrysidales,Noelaerhabdaceae,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ASV_138295,1,1.050850e-69,307,81.10,Not_Found,Not_Found,Hecatonema maculans mitochondrial partial COI ...,291,Eukaryota,NaN,unknown,Phaeophyceae,Ectocarpales,Chordariaceae,g_,s_
ASV_138351,1,3.274330e-76,331,82.99,Aphanomyces,Aphanomyces iridis,Aphanomyces iridis voucher CBS52487 cytochrome...,288,Eukaryota,NaN,Oomycota,unknown,Saprolegniales,Saprolegniaceae,g_,s_
ASV_138768,1,1.105320e-75,329,82.27,Not_Found,Not_Found,Laurencia sp. ARS-2010 voucher ARS03166 cytoch...,299,Eukaryota,NaN,NaN,NaN,NaN,NaN,g_,s_


In [88]:
'''

#Remove PhiX OTUs
list_DUP = (taxon_A, taxon_B)
#Number of OTUs without duplicates
for i in range(len(list_DUP)):
    y=list(set(list_DUP[i]))   #Remove duplicate entries from each list
    print(taxon[i], ' number of OTUs: ', len(y))    #Get number of OTUs which hit each taxon

#Get total number of otus and reads with PhiX and phiX; list of unique ASVs with PhiX hits
taxons = taxon_A + taxon_B
taxons= list(set(taxons))
print('Total Number OTUs with PhiX hits:',len(taxons))   #Number of OTUS

if len(taxons)== 0 :
    print('No Phix OTUs removed')
else:
    #remove these ASVs from the OTU table
    df = otu_tab.copy()
    df.reset_index(inplace=True)
    df=df.loc[df['ASV'].isin(taxons)==False]
    otu_tab = df.copy()

#ASV taxa table taken from Blast XML files will only contain ASVs with blast hits
#Ideally want to assign taxonomy from old table for those hits that weren't changed by this filtering
#They should all be 'no_hit'

#Join  new taxonomy and old taxonomy table together; check 'no_hit' for all missing taxa
df= taxa_lim.copy()
df['New']=1
df=pd.concat([df, taxa_tab], axis=1, keys=['new','old'],sort=False)
df=df.sort_values(('new', 'New'))
df=df.loc[df[('new', 'New')]!=1]
print('Unique Kingdom IDs from taxa without a Blast hit: ')
print("Should just be 'no_hit':")
print(df[('old', 'Kingdom')].unique())

#Since they're all no_hit, assign any missing values as 'no_hit'
df=pd.concat([taxa_lim, otu_tab], axis=1,sort=False)
df=df.fillna('no_hit')
df.index.name ='ASV'

#export 'OTU_taxa_table_all.csv'
outname = directory + 'Filtered_ASV_taxa_table_all.csv'
print(outname)
df.to_csv(outname)

print('Finished Filtering Taxonomy by BLAST hits')'''

'\n\n#Remove PhiX OTUs\nlist_DUP = (taxon_A, taxon_B)\n#Number of OTUs without duplicates\nfor i in range(len(list_DUP)):\n    y=list(set(list_DUP[i]))   #Remove duplicate entries from each list\n    print(taxon[i], \' number of OTUs: \', len(y))    #Get number of OTUs which hit each taxon\n\n#Get total number of otus and reads with PhiX and phiX; list of unique ASVs with PhiX hits\ntaxons = taxon_A + taxon_B\ntaxons= list(set(taxons))\nprint(\'Total Number OTUs with PhiX hits:\',len(taxons))   #Number of OTUS\n\nif len(taxons)== 0 :\n    print(\'No Phix OTUs removed\')\nelse:\n    #remove these ASVs from the OTU table\n    df = otu_tab.copy()\n    df.reset_index(inplace=True)\n    df=df.loc[df[\'ASV\'].isin(taxons)==False]\n    otu_tab = df.copy()\n\n#ASV taxa table taken from Blast XML files will only contain ASVs with blast hits\n#Ideally want to assign taxonomy from old table for those hits that weren\'t changed by this filtering\n#They should all be \'no_hit\'\n\n#Join  new taxono

## Parse results from MZG blastn search

In [54]:
# from Bio.Blast import NCBIXML

# Parse MZGB xml file
files = ['/Users/kpitz/Projects/custom_blast/Flyer2018_COI_MZG_COI_blastn_061223.xml']

# just keep top 4 hits for each query

# Parse through XML files
xml_dfs = []  #will store results from each xml file
for filename in files:
    print('Blast XML file: ', filename)
    query=[]
    hit_dict ={}  #Dictionary to store hits
    result_handle =open(filename)
    #Phix names to filter out - added for diversity in sequencing process
    taxon = ['PhiX', 'phiX']
    #parse blast records
    blast_records= NCBIXML.parse(result_handle)
    for blast_record in blast_records:
        query = blast_record.query
        hit_ID=0 #hit counter
        for alignment in blast_record.alignments:
            hit_ID+=1
            if hit_ID >4:
                continue
            key = (query, hit_ID)
            hsp = alignment.hsps[0] #only look at top hsp per alignment to genbank sequence
            #set limits on evalue and bitscore and % Identity
            per_iden = hsp.identities/float(hsp.align_length)
            per_iden = (per_iden *100)
            per_iden = round(per_iden, 2)
            evalue = hsp.expect
            bitscore = int(hsp.score)
            #alignment length
            align_len = hsp.align_length
            value = (evalue, bitscore, per_iden, alignment.hit_def, align_len)
            hit_dict[key]=value
    print ('Done parsing ', filename)
    #Save Blast Results to file
    df = pd.DataFrame(hit_dict)
    df=df.T
    df.columns = ['eval','bitscore', '%ID', 'hit_def', 'align_len']
    df.reset_index(inplace=True)
    df = df.rename(columns={'level_0': 'ASV', 'level_1': 'Hit_number'})
    xml_dfs.append(df)



Blast XML file:  /Users/kpitz/Projects/custom_blast/Flyer2018_COI_MZG_COI_blastn_061223.xml
Done parsing  /Users/kpitz/Projects/custom_blast/Flyer2018_COI_MZG_COI_blastn_061223.xml


## Limit by percent ID and just keep one hit per query

In [63]:
df = pd.concat(xml_dfs, axis=1)
df = df.loc[df['%ID']>=80]
df = df.drop_duplicates('ASV', keep='first')
#df = df.loc[df['ASV']=='ASV_3']
df.to_csv('/Users/kpitz/Documents/test.csv')
df = df.set_index('ASV')
MZgb = df.copy()
df

,Hit_number,eval,bitscore,%ID,hit_def,align_len
ASV,,,,,,
ASV_1,1,0.0,226,98.72,AB649198__Emiliania_huxleyi MZGdb-v2023-m04-...,235
ASV_2,1,0.0,228,98.73,AB649198__Emiliania_huxleyi MZGdb-v2023-m04-...,237
ASV_3,2,0.0,98,86.23,MG669420__Thysanopoda_orientalis MZGdb-v2023...,167
ASV_4,1,0.0,231,99.16,AB649198__Emiliania_huxleyi MZGdb-v2023-m04-...,237
ASV_5,1,0.0,234,99.58,AB649198__Emiliania_huxleyi MZGdb-v2023-m04-...,237
...,...,...,...,...,...,...
ASV_138196,1,0.0,234,99.58,AB649198__Emiliania_huxleyi MZGdb-v2023-m04-...,237
ASV_138288,1,0.0,120,80.62,KJ960337__Apoglossum_ruscifolium MZGdb-v2023...,289
ASV_138768,1,0.0,135,81.88,MT760752__Phaeocystis_globosa MZGdb-v2023-m0...,298


## Join MZG results with genbank results
- Make columns to show when genera or species results match between methods

In [110]:
# join with MEGAN:
df = tophit_Ftaxonomy.reset_index()
df = df.rename(columns={'index':'ASV'})
df = df.set_index('ASV')
df = pd.concat([MZgb, df], axis=1, keys=['MZGdb', 'genbank'])
df.to_csv(directory + 'MZGdb_Genbank_megan_comparison.csv')
# Now compare
#Genus
df['genus_match'] = df.apply(lambda x: str(x['genbank'].Genus) in str(x['MZGdb'].hit_def), axis=1)
df.loc[df['genbank']['Genus'].isna(), 'genus_match'] = False
df = df.sort_values('genus_match')
# Species
df['hit_sp_def'] = df['MZGdb']['hit_def'].str.replace('_',' ')
df['sp_match'] = df.apply(lambda x: str(x['genbank'].Species) in str(x.hit_sp_def), axis=1)
df.loc[df['genbank']['Species'].isna(), 'sp_match'] = False
df = df.sort_values('sp_match')

# drop unecessary columns
df = df.drop(['hit_sp_def'], axis=1)
df = df.loc[(df['sp_match']==False) & (df['genus_match']==False)]
df = df.sort_values([('MZGdb', 'bitscore'), ('genbank', 'bitscore')], ascending=False)
df.to_csv(directory + 'MZGdb_Genbank_megan_diffs.csv')

df

MZGdb                       \
          Hit_number eval bitscore    %ID   
ASV                                         
ASV_1623         1.0  0.0      316  100.0   
ASV_2672         1.0  0.0      313  100.0   
ASV_537          1.0  0.0      313  100.0   
ASV_14705        1.0  0.0      313  100.0   
ASV_23324        1.0  0.0      313  100.0   
...              ...  ...      ...    ...   
ASV_16621        NaN  NaN      NaN    NaN   
ASV_26530        NaN  NaN      NaN    NaN   
ASV_25986        NaN  NaN      NaN    NaN   
ASV_44269        NaN  NaN      NaN    NaN   
ASV_65214        NaN  NaN      NaN    NaN   

                                                                        \
                                                     hit_def align_len   
ASV                                                                      
ASV_1623   CNIDC447-16__Muggiaea_atlantica   MZGdb_v2023-...       316   
ASV_2672   NC_013935__Pycnococcus_provasolii   MZGdb-v202...       313   
ASV_537    JN050306__Pseudonitzschia_cuspidata   MZGdb-v2...       313   
ASV_14705  HM473956__Strongylocentrotus_fragilis   MZGdb-...       313   
ASV_23324  MW446643__Stenella_coeruleoalba   MZGdb-v2023-...       313   
...                                                      ...       ...   
ASV_16621                                                NaN       NaN   
ASV_26530                                                NaN       NaN   
ASV_25986                                                NaN       NaN   
ASV_44269                                                NaN       NaN   
ASV_65214                                                NaN       NaN   

             genbank                                  ...             \
          Hit_number           eval bitscore     %ID  ...     Domain   
ASV                                                   ...              
ASV_1623         2.0  1.951310e-123    505.0   92.06  ...  Eukaryota   
ASV_2672         1.0  1.481300e-156    626.0  100.00  ...  Eukaryota   
ASV_537          1.0  1.481300e-156    626.0  100.00  ...  Eukaryota   
ASV_14705        1.0  1.481300e-156    626.0  100.00  ...  Eukaryota   
ASV_23324        1.0  1.481300e-156    626.0  100.00  ...  Eukaryota   
...              ...            ...      ...     ...  ...        ...   
ASV_16621        1.0   8.357320e-06     70.0   91.11  ...        NaN   
ASV_26530        1.0   8.312630e-06     70.0   95.00  ...        NaN   
ASV_25986        1.0   8.312630e-06     70.0   91.11  ...        NaN   
ASV_44269        1.0   8.848930e-06     70.0   95.00  ...        NaN   
ASV_65214        1.0   8.357320e-06     70.0   91.11  ...        NaN   

                                                                             \
                 Kingdom           Phylum              Class          Order   
ASV                                                                           
ASV_1623         Metazoa         Cnidaria           Hydrozoa  Siphonophorae   
ASV_2672   Viridiplantae      Chlorophyta            unknown        unknown   
ASV_537              NaN  Bacillariophyta  Bacillariophyceae  Bacillariales   
ASV_14705        Metazoa    Echinodermata         Echinoidea    Camarodonta   
ASV_23324        Metazoa         Chordata           Mammalia   Artiodactyla   
...                  ...              ...                ...            ...   
ASV_16621            NaN              NaN                NaN            NaN   
ASV_26530            NaN              NaN                NaN            NaN   
ASV_25986            NaN              NaN                NaN            NaN   
ASV_44269            NaN              NaN                NaN            NaN   
ASV_65214            NaN              NaN                NaN            NaN   

                                                                               \
                         Family             Genus                     Species   
ASV                                                                  

In [106]:
## compare results
#Genus
df['genus_match'] = df.apply(lambda x: str(x['genbank'].Genus) in str(x['MZGdb'].hit_def), axis=1)
df.loc[df['genbank']['Genus'].isna(), 'genus_match'] = False
df = df.sort_values('genus_match')
# Species
df['hit_sp_def'] = df['MZGdb']['hit_def'].str.replace('_',' ')
df['sp_match'] = df.apply(lambda x: str(x['genbank'].Species) in str(x.hit_sp_def), axis=1)
df.loc[df['genbank']['Species'].isna(), 'sp_match'] = False
df = df.sort_values('sp_match')

# drop unecessary columns
df = df.drop(['hit_sp_def'], axis=1)
df.to_csv('/Users/kpitz/Documents/test.csv')
df


MZGdb                       \
           Hit_number eval bitscore    %ID   
ASV                                          
ASV_1             1.0  0.0      226  98.72   
ASV_125642        1.0  0.0      149  83.79   
ASV_125681        1.0  0.0      114  98.33   
ASV_125687        1.0  0.0      161  85.37   
ASV_130778        1.0  0.0      210   96.2   
...               ...  ...      ...    ...   
ASV_907           1.0  0.0      302  99.35   
ASV_16879         1.0  0.0      305  99.68   
ASV_3893          1.0  0.0      305  99.68   
ASV_24535         1.0  0.0      287  97.73   
ASV_5228          1.0  0.0      302  99.35   

                                                                         \
                                                      hit_def align_len   
ASV                                                                       
ASV_1       AB649198__Emiliania_huxleyi   MZGdb-v2023-m04-...       235   
ASV_125642  AB948162__Skeletonema_menzelii   MZGdb-v2023-m...       290   
ASV_125681  KHBC002-13__Pandea_rubra   MZGdb_v2023-m04-01 ...       120   
ASV_125687  LR880960__Eurypon_clavigerum   MZGdb-v2023-m04...       287   
ASV_130778  AB649198__Emiliania_huxleyi   MZGdb-v2023-m04-...       237   
...                                                       ...       ...   
ASV_907     MG136783__Leuckartiara_longicalcar   MZGdb-v20...       308   
ASV_16879   MN745808__Ctenocalanus_vanus   MZGdb-v2023-m04...       308   
ASV_3893    KC287601__Clausocalanus_paululus   MZGdb-v2023...       308   
ASV_24535   KX650375__Oncaea_scottodicarloi   MZGdb-v2023-...       308   
ASV_5228    MG136783__Leuckartiara_longicalcar   MZGdb-v20...       308   

              genbank                                  ...             \
           Hit_number           eval bitscore     %ID  ...     Domain   
ASV                                                    ...              
ASV_1             1.0  7.533310e-154    616.0  100.00  ...  Eukaryota   
ASV_125642        6.0   5.438750e-86    366.0   87.31  ...  Eukaryota   
ASV_125681        1.0   3.739310e-50    235.0   99.17  ...  Eukaryota   
ASV_125687        1.0  2.062680e-110    457.0   91.41  ...  Eukaryota   
ASV_130778        1.0  3.426170e-120    492.0   92.69  ...  Eukaryota   
...               ...            ...      ...     ...  ...        ...   
ASV_907           1.0  3.902350e-151    606.0   99.35  ...  Eukaryota   
ASV_16879         1.0  3.203250e-152    611.0   99.68  ...  Eukaryota   
ASV_3893          1.0  3.203250e-152    611.0   99.68  ...  Eukaryota   
ASV_24535         1.0  4.452580e-144    581.0   97.73  ...  Eukaryota   
ASV_5228          1.0  3.902350e-151    606.0   99.35  ...  Eukaryota   

                                                                       \
            Kingdom           Phylum         Class              Order   
ASV                                                                     
ASV_1           NaN       Haptophyta       unknown     Isochrysidales   
ASV_125642      NaN  Bacillariophyta           NaN                NaN   
ASV_125681  Metazoa         Chordata      Mammalia           Primates   
ASV_125687  Metazoa         Porifera  Demospongiae         Axinellida   
ASV_130778      NaN       Haptophyta       unknown     Isochrysidales   
...             ...              ...           ...                ...   
ASV_907     Metazoa         Cnidaria      Hydrozoa      Anthoathecata   
ASV_16879   Metazoa       Arthropoda   Hexanauplia          Calanoida   
ASV_3893    Metazoa       Arthropoda   Hexanauplia          Calanoida   
ASV_24535   Metazoa       Arthropoda   Hexanauplia  Poecilostomatoida   
ASV_5228    Metazoa         Cnidaria      Hydrozoa      Anthoathecata   

                                                                       \
                      Family          Genus                   Species   
ASV                                                                     
ASV_1       Noelaerhabdaceae            NaN            

In [78]:
# join with MEGAN:
df = pd.concat([MZgb, megan], axis=1)
#df['genus_match'] = ''
df['genus_match'] = df.apply(lambda x: str(x.Genus) in str(x.hit_def), axis=1)
df.loc[df['Genus'].isna(), 'genus_match'] = False
df = df.sort_values('genus_match')
# Species
df['hit_sp_def'] = df['hit_def'].str.replace('_',' ')
df['sp_match'] = df.apply(lambda x: str(x.Species) in str(x.hit_sp_def), axis=1)
df.loc[df['Species'].isna(), 'sp_match'] = False
df = df.sort_values('sp_match')

# drop unecessary columns
df = df.drop(['taxonomy', 'hit_sp_def'], axis=1)
df.to_csv('/Users/kpitz/Documents/test.csv')
df

,Hit_number,eval,bitscore,%ID,hit_def,align_len,Domain,Kingdom,Phylum,Class,Order,Family,Genus,Species,genus_match,sp_match
ASV,,,,,,,,,,,,,,,,
ASV_1,1.0,0.0,226,98.72,AB649198__Emiliania_huxleyi MZGdb-v2023-m04-...,235,Eukaryota,NaN,Haptophyta,unknown,Isochrysidales,Noelaerhabdaceae,NaN,NaN,False,False
ASV_2745,1.0,0.0,47,84.27,EU982725__Cadlina_rumia MZGdb-v2023-m04-01 (...,89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
ASV_3142,1.0,0.0,296,98.08,MW691182__Mesonerilla_roscovita MZGdb-v2023-...,312,Eukaryota,Metazoa,Chordata,Mammalia,Primates,Hominidae,Homo,Homo sapiens,False,False
ASV_3136,1.0,0.0,124,81.03,CFAD137-11__Pagurus_villosus MZGdb_v2023-m04...,290,Eukaryota,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Pleurotaceae,Pleurotus,NaN,False,False
ASV_2734,1.0,0.0,155,98.76,HQ646565__Octactis_octonaria MZGdb-v2023-m04...,161,Eukaryota,NaN,unknown,Dictyochophyceae,Dictyochales,unknown,Dictyocha,Dictyocha speculum,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ASV_98427,1.0,0.0,133,81.29,MT760752__Phaeocystis_globosa MZGdb-v2023-m0...,310,Eukaryota,NaN,Haptophyta,unknown,Phaeocystales,Phaeocystaceae,Phaeocystis,Phaeocystis globosa,True,True
ASV_27159,1.0,0.0,308,100.0,MN745887__Ctenocalanus_vanus MZGdb-v2023-m04...,308,Eukaryota,Metazoa,Arthropoda,Hexanauplia,Calanoida,Calanidae,Ctenocalanus,Ctenocalanus vanus,True,True
ASV_98421,1.0,0.0,254,94.16,MT760738__Phaeocystis_globosa MZGdb-v2023-m0...,308,Eukaryota,NaN,Haptophyta,unknown,Phaeocystales,Phaeocystaceae,Phaeocystis,Phaeocystis globosa,True,True


In [ ]:
#Export Files
# df = pd.concat([meta[['sample_name']], otu_table.T], axis=1, sort=False)
#rename columns as sample_names
# df.set_index('sample_name', inplace = True)
#Join with taxonomy
samples = list(df.T)
print('Number of Samples in OTU table:', len(samples))
df = pd.concat([taxa_tab, df.T], axis=1, sort=False)
df.fillna('no_hit', inplace=True)
#sort by ASV number - relates to abundance of ASV
df['OTU_num'] = df.index.str.split('_').str[-1].astype(int)
df=df.sort_values('OTU_num')
df.drop('OTU_num', inplace=True, axis=1)
#Make index names
df.index.name='ASV'
#export 'OTU_taxa_table_all.csv'
df.to_csv(directory+'ASV_taxa_table_all.csv')
#export taxa table
taxa_tab_all = df[levels].copy()
taxa_tab_all.index.name='#ASV'  #Biom conversion needs # in index header
taxa_tab_all.to_csv(directory+'Taxa_table.tsv', sep='\t')
#export otu table
otu_tab_all = df[samples].copy()
otu_tab_all.to_csv(directory+'ASV_table.tsv', sep='\t')
#Print some basic stats of total reads and ASVs
df['Total_Reads'] = df[samples].sum(axis=1)
df['Total_ASVs']=1
df=df.groupby(['Kingdom','Phylum']).sum()
print('XXXXXX Summary of Total Reads and ASVs by Kingdom and Phylum')
print(df[['Total_Reads','Total_ASVs']])
print('XXXXXX End Summary')